In [2]:
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib
import xgboost as xgb
from xgboost_model import XgbModel
from hyperopt import hp

%matplotlib inline
import mpld3
#mpld3.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
df = pd.read_pickle('data/formated_train_data_7l.pkl')

print df.shape
df.head(20)

(2652328, 13)


,x1,x2,x3,x4,x5,x6,x7,x8,point_id,point_label,probe_id,probe_label,sim
0,2732.521729,0.000000,0.000000,0,0,0,0,0,41004,13721,7490,13721,1
1,2544.442139,0.000000,0.000000,0,0,0,0,0,29667,26694,7490,13721,0
2,2538.058105,0.000000,0.000000,0,0,0,0,0,130431,7610,7490,13721,0
3,2537.544189,0.000000,0.000000,0,0,0,0,0,116675,65463,7490,13721,0
4,2533.375488,0.000000,0.000000,0,0,0,0,0,127651,17181,7490,13721,0
5,2527.897705,0.000000,0.000000,0,0,0,0,0,69335,51818,7490,13721,0
6,2522.609863,0.000000,0.000000,0,0,0,0,0,128410,57058,7490,13721,0
7,0.000000,4321.461914,0.000000,0,0,0,0,0,78881,5487,7490,13721,0
8,0.000000,4164.022461,0.000000,0,0,0,0,0,41004,13721,7490,13721,1
9,0.000000,3976.418945,0.000000,0,0,0,0,0,78720,37921,7490,13721,0


## Searching good hyper-params

In [9]:
# defining the hyper-parameters space to explore
space = {
             'n_estimators' : hp.quniform('n_estimators', 500, 1000, 1),
             'eta' : hp.uniform('eta', 0.025, 0.5),
             'max_depth' : hp.quniform('max_depth', 1, 15, 1),
             'min_child_weight' : hp.quniform('min_child_weight', 1, 6, 1),
             'subsample' : hp.uniform('subsample', 0.5, 1),
             'gamma' : hp.uniform('gamma', 0.5, 1),
             'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1),
             'eval_metric': 'auc',
             'objective': 'binary:logistic',
             'nthread' : 6,
             'silent' : 0
             }

# separating into train and test set in a stratified way
df_train, df_test = train_test_split(df, test_size=0.5, random_state=42, stratify=df['sim'])

print df_train.shape, df_test.shape
print len(df[df['sim']==1])/float(len(df))
print len(df_train[df_train['sim']==1])/float(len(df_train)), len(df_test[df_test['sim']==1])/float(len(df_test)) 

# defining the stratified kfold for cross-validation
skf = StratifiedKFold(df_train['sim'], n_folds=3, random_state=42)


features = ['x'+str(i+1) for i in range(8)]
target = 'sim'


(1326164, 13) (1326164, 13)
0.200935555482
0.200935932509 0.200935178455


In [10]:
model = XgbModel()

model.optimize(space, df_train, features, target, skf, output_file='output_hyperopt_xgboost.txt')

mail.sendMail('maxime.kubryk@gmail.com','script xgboost ended', 'cool !!')

Index error
Training with params : 
{'colsample_bytree': 0.9903820991923078, 'silent': 0, 'eval_metric': 'auc', 'nthread': 6, 'min_child_weight': 2.0, 'n_estimators': 643.0, 'subsample': 0.8482345927989308, 'eta': 0.22597556855911893, 'objective': 'binary:logistic', 'max_depth': 9.0, 'gamma': 0.8597344848927815}
accuracy: 0.950648674939
accuracy: 0.950494847926
accuracy: 0.950938120682
	Score -0.950693881182


Training with params : 
{'colsample_bytree': 0.8339812047040367, 'silent': 0, 'eval_metric': 'auc', 'nthread': 6, 'min_child_weight': 4.0, 'n_estimators': 873.0, 'subsample': 0.5530324529773664, 'eta': 0.15514492046967743, 'objective': 'binary:logistic', 'max_depth': 7.0, 'gamma': 0.6923529515220681}
accuracy: 0.950818337085
accuracy: 0.950648674939
accuracy: 0.950999199193
	Score -0.950822070405


Training with params : 
{'colsample_bytree': 0.7533430554699614, 'silent': 0, 'eval_metric': 'auc', 'nthread': 6, 'min_child_weight': 4.0, 'n_estimators': 529.0, 'subsample': 0.7533630

KeyboardInterrupt: 